In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-08-05 17:21:05,134 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-05 17:21:05,176 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-05 17:21:05,229 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-05 17:21:05,231 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-05 17:21:05,241 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-05 17:21:05,242 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-05 17:21:05,245 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-05 17:21:05,249 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-05 17:21:0

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "fill_rate_schema" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-05 17:21:05,458 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-05 17:21:05,490 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [5]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

transform = Transform()

In [6]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

![Fill Rate Schema](assets/FillRateSchema.png)



### Transformation

In [152]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
# run_filter.append(dict(partition="run_id", comparison="==", values=[1]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

2019-08-05 20:47:29,465 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


In [153]:
import os

import numpy as np
import pandas as pd

pd.options.display.max_columns=999

df.status_date = df.status_date.str[:8].astype(str)
df.ref_date = df.ref_date.str[:8].astype(str)

df.status_date = pd.to_datetime(df.status_date, infer_datetime_format=True, errors='coerce')
df.ref_date = pd.to_datetime(df.ref_date, infer_datetime_format=True, errors='coerce')

In [154]:
os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config.loc[:,'statusCode'] = status_config.statusCode.str.upper()
status_config.loc[:,'subStatus'] = status_config.subStatus.str.upper()
status_config.loc[:,'integrichain_sub_status'] = status_config.integrichain_sub_status.str.upper()
status_config.loc[:,'integrichain_status'] = status_config.integrichain_status.str.upper()
status_config.loc[:,'Patient_Journey_Hierarchy'] = status_config.Patient_Journey_Hierarchy.str.upper()

status_config = status_config.rename(columns={'statusCode':'status_code','subStatus':'sub_status'})

df.sub_status = df.sub_status.str.replace('PRESCRIBERHOLD','PRESCRIBER HOLD')

df = pd.merge(df,status_config,on=['status_code','sub_status'])

df = df[['rec_date', 'pharm_code', 'pharm_npi', 'transtype', 'pharm_transaction_id', 'trans_seq', 'ref_source', 'ref_date', 'program_id', 'pharmacy_id', 'pat_last_name', 'pat_first_name', 'pat_dob', 'pat_gender', 
         'pat_addr1', 'pat_addr2', 'pat_city', 'pat_state', 'pat_zip', 'dx1_code', 'dx2_code', 'status_date', 'status_code', 'sub_status', 'integrichain_status','integrichain_sub_status', 'Patient_Journey_Hierarchy', 
         'pres_last_name', 'pres_first_name', 'pres_addr1', 'pres_addr2', 'pres_city', 'pres_state', 'pres_zip', 'pres_phone', 'pres_npi', 'pres_dea', 'facility_name', 'rxdate', 'rxnumber', 'rxrefills', 'rxfill', 
         'refill_remaining', 'prev_disp', 'rx_ndc_number', 'medication', 'quantity', 'day_supply', 'ship_date', 'ship_carrier', 'shiptracking_num', 'ship_location', 'ship_address', 'ship_city', 'ship_state', 'ship_zip', 
         'has_medical', 'primary_coverage_type', 'primary_payer_name', 'primary_payer_type', 'secondary_coverage_type', 'secondary_payer_name', 'secondary_payer_type', 'plan_paid_amt', 'pat_copay', 'copay_assist_amount', 
         'oth_payer_amt', 'xfer_pharmname', 'msa_patient_id', 'msa_patient_bmap', '__metadata_run_timestamp', '__metadata_app_version', '__metadata_output_contract', '__metadata_transform_timestamp', '__metadata_run_id']]

In [155]:
sort_columns = ['msa_patient_id', 'pharm_code', 'medication', 'status_date']
col_ids = sort_columns[:-1]
status_date = sort_columns[-1]
transaction_id = ['pharm_transaction_id']
payer_info = ['ref_source','pres_first_name','pres_last_name','pres_state','primary_payer_name','primary_payer_type']
status_info = ['integrichain_status','integrichain_sub_status','Patient_Journey_Hierarchy']
shipment = 'SHIPMENT'
transfered = 'TRANSFERED'
cancelled = 'CANCELLED'

In [156]:
df = df.sort_values(sort_columns,ascending=[True, True, True, True])

In [157]:
max_date_df = (
    df
    .groupby(col_ids)
    .status_date
    .max()
    .reset_index(drop=False)
    .rename(columns={status_date:'max_status_date'})
)

In [158]:
df = pd.merge(df,max_date_df,on=col_ids)

In [178]:
df.pharm_transaction_id = df.pharm_transaction_id.str.extract(r'(\d+$)')

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,000000
7,000000
8,000000
9,000000


In [97]:
df['max_date'] = np.where(df[status_date] == df['max_status_date'],1,0)
df['max_transaction_id'] = np.where(df[transaction_id] == df[transaction_id].max(),1,0)

In [98]:
df = df[df.max_date == 1][col_ids + transaction_id + [status_date] + payer_info + status_info].drop_duplicates()

In [100]:
df[df.msa_patient_id == '4500000']

,msa_patient_id,pharm_code,medication,pharm_transaction_id,status_date,ref_source,pres_first_name,pres_last_name,pres_state,primary_payer_name,primary_payer_type,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy
25928,4500000,CVS,ILUMYA,901268010120190502000000,2019-05-02,DIRECT,NICHOLE,HART,MD,None,None,CANCELLED,PATIENT END,PATIENT
25931,4500000,CVS,ILUMYA,901268012820190502000000,2019-05-02,DIRECT,NICHOLE,REAGLE,MD,None,None,PENDING,APPEAL,BV/PA
25937,4500000,CVS,ILUMYA SD PFS,188371701,2019-05-06,DIRECT,NICHOLE,HART,MD,None,COMMERICAL,ACTIVE,SHIPMENT,FULFILLMENT


In [88]:
df['filled'] = np.where((df.integrichain_sub_status == shipment),1,0)
df['transferred'] = np.where((df.Patient_Journey_Hierarchy == transfered),1,0)
df['cancelled'] = np.where((df.integrichain_status == cancelled) & ~(df.Patient_Journey_Hierarchy == transfered),1,0)
df['stillopen'] = np.where((df.filled == 0) & (df.transferred == 0) & (df.cancelled == 0),1,0)

In [89]:
df[df.msa_patient_id == '4500000']

,msa_patient_id,pharm_code,medication,status_date,ref_source,pres_first_name,pres_last_name,pres_state,primary_payer_name,primary_payer_type,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy,filled,transferred,cancelled,stillopen
25928,4500000,CVS,ILUMYA,2019-05-02,DIRECT,NICHOLE,HART,MD,None,None,CANCELLED,PATIENT END,PATIENT,0,0,1,0
25931,4500000,CVS,ILUMYA,2019-05-02,DIRECT,NICHOLE,REAGLE,MD,None,None,PENDING,APPEAL,BV/PA,0,0,0,1
25937,4500000,CVS,ILUMYA SD PFS,2019-05-06,DIRECT,NICHOLE,HART,MD,None,COMMERICAL,ACTIVE,SHIPMENT,FULFILLMENT,1,0,0,0


In [46]:
import unittest

def no_multiple_types(df):
    return df[df.filled + df.transferred + df.cancelled + df.stillopen != 1].shape[0] == 0

def duplicates_exist(df):
    return df[col_ids].drop_duplicates().shape[0] == df.shape[0]

In [47]:
class TestNotebook(unittest.TestCase):
    
    def test_no_multiple_types(self):
        self.assertEqual(no_multiple_types(final_dataframe),True)
        
    def test_no_multiple_types(self):
        self.assertEqual(duplicates_exist(final_dataframe),True)
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_no_multiple_types (__main__.TestNotebook) ... FAIL

FAIL: test_no_multiple_types (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-47-f087ce2b1092>", line 7, in test_no_multiple_types
    self.assertEqual(duplicates_exist(test_df),True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 1 test in 0.008s

FAILED (failures=1)


In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()